---
## START : Configuration Area 

In [1]:
import tensorflow as tf
import os
import argparse as _argparse
from PIL import Image
import numpy as np
import collections
import copy
import random
from tqdm import tqdm
print('=== Finish ===')

=== Finish ===


In [2]:
train_dir = os.path.join('..','data','image_data','train_data') # Train dataset directory
dev_dir   = os.path.join('..','data','image_data','dev_data')  # validation dataset directory
test_dir  = os.path.join('..','data','image_data','test_data')
processed_dir = os.path.join('..','data','processed_data')      # processed data 

target = "yongho_BG_with_attention"
target_dir = os.path.join('..','data','research_data',target)
model_ckpt_dir = os.path.join('..','data','research_data',target,'model_ckpt')
train_log_dir = os.path.join('..','data','research_data',target,'train_log')
result_dir = os.path.join('..','data','research_data',target, 'result')

os.makedirs(processed_dir) if not os.path.exists(processed_dir) else 0
os.makedirs(target_dir) if not os.path.exists(target_dir) else 0
os.makedirs(model_ckpt_dir) if not os.path.exists(model_ckpt_dir) else 0
os.makedirs(train_log_dir) if not os.path.exists(train_log_dir) else 0
os.makedirs(result_dir) if not os.path.exists(result_dir) else 0

train_record_file = os.path.join(processed_dir,'train.tfrecord')
dev_record_file = os.path.join(processed_dir,'dev.tfrecord')
test_record_file = os.path.join(processed_dir,'test.tfrecord')
print('=== Finish ===')

=== Finish ===


In [3]:


################## Only JupyterNotebook
flags.DEFINE_string("f",'','')
flags.DEFINE_string("kernel",'',"")
##################
print('=== Finish ===')

=== Finish ===


## END : Configuration Area 
---

---
## START : Prepro

In [4]:
def list_check(src, dst):
    if len(src) != len(dst):
        return False
    for idx, s in enumerate(src):
        if s != dst[idx]:
            return False
    return True
        
def filtered_v01(image):
    return None

def detect_object(image):
    # input : checked image
    pix = np.array(image)
    print('np.shape()')
    ## [ 세로 ] [ 가로 ] [ 채널 ]
    print(np.shape(pix))
    # print(len(pix))
    # print(len(pix[0])) # [height][width][channel]
    # print(len(pix[0][0]))
    width  = [ [0, 0], [0, 0] ]   # width's  [start point[x, y] , end point[x,y]]
    height = [ [0, 0], [0, 0] ]   # height's [start point[x, y] , end point[x, y]]
    
    # Calculating Width 
    for i in range( len(pix) ): # 
        start = [0, 0]
        keep = 0
        out = 0
        for j in range( len(pix[0]) ):
            if list_check(pix[i][j],[255,0,0]) and list_check(start,[0,0]):#(start == [0, 0]) && 
                start = [i, j]
                keep = 1
            if keep > 0 and list_check(pix[i][j],[255, 0, 0]):
                end = [i, j]
                keep += 1
            if keep > 50 and not list_check(pix[i][j], [255, 0, 0]):         
                width[0] = start
                width[1] = end
                out = 1
                break
        if out == 1:
            break
            
    for j in range( len(pix[0]) ):
        start = [0,0]
        keep = 0
        out = 0
        for i in range( len(pix) ):
            if list_check(start,[0, 0]) and list_check(pix[i][j], [255,0,0]):
                start = [i, j]
                keep = 1
            if keep > 0 and list_check(pix[i][j], [255, 0, 0]):
                end = [i, j]
                keep += 1
            if keep > 50 and not list_check(pix[i][j], [255, 0, 0]):         
                height[0] = start
                height[1] = end
                out = 1
                break
        if out == 1:
            break
    # { loc : (y, x), size : (width, height)}
    return width[0] + list((width[1][1] - width[0][1], height[1][0] - height[0][0]))    

def get_plus_info(obj_info, img_info):
    plus_info = {}

    plus_info['left']  = random.randrange(0, obj_info['x'])
    plus_info['right'] = random.randrange(0, img_info['w'] - (obj_info['x'] + obj_info['w']))
    plus_info['down']  = random.randrange(0, img_info['h'] - (obj_info['y'] + obj_info['h']))
    plus_info['up']    = random.randrange(0, obj_info['y'])

    return plus_info

def setting_etc(image_pix, plus_info, t_obj_info):
    output_pix = copy.deepcopy(image_pix)
    output_pix = output_pix[t_obj_info['y'] - plus_info['up'] : t_obj_info['y'] + t_obj_info['h'] + plus_info['down'], :]
    output_pix = output_pix[:, t_obj_info['x'] - plus_info['left'] : t_obj_info['x'] + t_obj_info['w'] + plus_info['right']]
    t_obj_info['x'] = plus_info['left']    
    t_obj_info['y'] = plus_info['up']

    return output_pix, t_obj_info

In [5]:
def augment_image(dataset, image, obj_info, num, cnt):

    possible = {'up':0,'down':0,'left':0,'right':0} # [상, 하, 좌, 우]
    image_pix = np.array(image)
    img_info = {'x':0, 'y':0, 'w':len(image_pix[0]), 'h':len(image_pix)}
    
    min_gen = 150
    # 최소 여백 부분을 100, 최소 생성 부분 50, 따라서 150으로 설정
    possible['up'] = 1 if obj_info['y'] > min_gen else 0
    possible['down'] = 1 if obj_info['y'] + obj_info['h'] + min_gen < img_info['h'] else 0
    possible['left'] = 1 if obj_info['x'] > min_gen else 0
    possible['right'] = 1 if obj_info['x'] + obj_info['w'] + min_gen < img_info['w'] else 0

    x_data_s = list()
    y_data_s = list()

    data_bundle = list()
    
    if possible['up']: # 위로만 생성 데이터
        for i in range(num):
            input_pix = None
            output_pix = None
            t_obj_info = copy.deepcopy(obj_info)
            plus_info = get_plus_info(t_obj_info, img_info)
            plus_info['up']    = t_obj_info['y']

            output_pix, t_obj_info = setting_etc(image_pix, plus_info, t_obj_info)
            input_pix = copy.deepcopy(output_pix)
            erase_area = random.randrange(50, t_obj_info['y'] - 100)
            input_pix[: erase_area, : ] = 0
            im_input = Image.fromarray(input_pix)
            im_output = Image.fromarray(output_pix)
            x_file = dataset + '/image_X_' + str(cnt) + '.bmp'
            y_file = dataset + '/image_Y_' + str(cnt) + '.bmp'
            im_input.save( x_file )
            im_output.save( y_file )
            data_bundle.append({'X': x_file, 'Y':y_file, 'obj_info' : t_obj_info, 'zero':'up', 'id':cnt})
            cnt += 1
    
    if possible['down']: # 아래로만 생성 데이터
        for i in range(num):
            input_pix = None
            output_pix = None
            t_obj_info = copy.deepcopy(obj_info)
            plus_info = get_plus_info(t_obj_info, img_info)
            plus_info['down'] = img_info['h'] - (t_obj_info['y'] + t_obj_info['h'])

            output_pix, t_obj_info = setting_etc(image_pix, plus_info, t_obj_info)
            input_pix = copy.deepcopy(output_pix)
            erase_area = random.randrange(50, len(output_pix) - 100 - t_obj_info['h'] - t_obj_info['y'])
            input_pix[ len(output_pix) - erase_area: , : ] = 0
            im_input = Image.fromarray(input_pix)
            im_output = Image.fromarray(output_pix)
            x_file = dataset + '/image_X_' + str(cnt) + '.bmp'
            y_file = dataset + '/image_Y_' + str(cnt) + '.bmp'
            im_input.save( x_file )
            im_output.save( y_file )
            data_bundle.append({'X': x_file, 'Y':y_file, 'obj_info' : t_obj_info, 'zero':'down', 'id':cnt})
            cnt += 1
    
    if possible['left']:# 왼으로만 생성 데이터
        for i in range(num):
            input_pix  = None
            output_pix = None
            t_obj_info = copy.deepcopy(obj_info)
            plus_info = get_plus_info(t_obj_info, img_info)
            plus_info['left'] = t_obj_info['x']

            output_pix, t_obj_info = setting_etc(image_pix, plus_info, t_obj_info)
            input_pix  = copy.deepcopy(output_pix)
            erase_area = random.randrange(50, t_obj_info['x'] - 100)
            input_pix[:, :erase_area ] = 0

            im_input  = Image.fromarray(input_pix)
            im_output = Image.fromarray(output_pix)
            x_file = dataset + '/image_X_' + str(cnt) + '.bmp'
            y_file = dataset + '/image_Y_' + str(cnt) + '.bmp'
            im_input.save( x_file )
            im_output.save( y_file )
            data_bundle.append({'X': x_file, 'Y':y_file, 'obj_info' : t_obj_info, 'zero': 'left', 'id':cnt})
            cnt += 1
    
    if possible['right']:
        for i in range(num):
            input_pix = None 
            output_pix = None
            t_obj_info = copy.deepcopy(obj_info)
            plus_info = get_plus_info(t_obj_info, img_info)
            plus_info['right'] = img_info['w'] - (t_obj_info['x'] + t_obj_info['w'])

            output_pix, t_obj_info = setting_etc(image_pix, plus_info, t_obj_info)
            input_pix = copy.deepcopy(output_pix)
            erase_area = random.randrange(50, len(output_pix[0]) - (t_obj_info['x'] + t_obj_info['w'] + 100) )
            input_pix[:, len(output_pix[0]) - erase_area : ] = 0
            im_input = Image.fromarray(input_pix)
            im_output = Image.fromarray(output_pix)
            x_file = dataset + '/image_X_' + str(cnt) + '.bmp'
            y_file = dataset + '/image_Y_' + str(cnt) + '.bmp'
            im_input.save( x_file )
            im_output.save( y_file )
            data_bundle.append({'X': x_file, 'Y':y_file, 'obj_info' : t_obj_info, 'zero':'right', 'id':cnt})
            cnt += 1
    return data_bundle, cnt

In [6]:
def make_tf_record(dataset, kind):
    cnt = 0
    data_list = os.listdir(dataset +'/original')
    tmp = list()
    input_tf = list()
    for data in data_list:
        if data.split("_")[1] != 'c':
            tmp.append(data)
    data_tuple = list()

    for data in tmp:
        c_data = data.split('_')
        c_data = c_data[0] +'_c_' + c_data[1]
        data_tuple.append( (data, c_data) )

    for data in data_tuple:
        print(data)
        origin = Image.open(os.path.join(dataset + '/original', data[0])) # Original Image
        chked  = Image.open(os.path.join(dataset + '/original', data[1])) # object detected Image        
        obj_info = detect_object(chked) # obj_info = [y, x, width, height]
        obj_info_ = {'x':obj_info[1],'y':obj_info[0],'w':obj_info[2],'h':obj_info[3]}
        obj_info = obj_info_
        ## 기본적인 생성방식, N 개의 image 를 random 방식으로 잘라서 data 를 augment
        N = 10 # 데이터 증강 개수
        tmp_tf, cnt = augment_image(dataset + '/augment', origin, obj_info, N, cnt) # 원래 이미지, 오브젝 정보, 데이터 증강 개수
            
        input_tf += tmp_tf

        N = 10
    return input_tf

In [7]:
def produce_tfrecord(examples):
    print('Generating train.tfrecord')
    writer = tf.python_io.TFRecordWriter(config.train_record_file)    
    for example in examples:
        print(example)
        img_x = Image.open(example['X'])
        img_y = Image.open(example['Y'])
        np_x = np.array(img_x, dtype=np.int32)
        np_y = np.array(img_y, dtype=np.int32)
        obj_info = example['obj_info']
        obj_info_ = np.array( [obj_info['x'], obj_info['y'], obj_info['w'], obj_info['h']], dtype=np.int32)
        
        if example['zero'] == 'up':
            np_x = np_x[:obj_info['y'],:]
            np_y = np_y[:obj_info['y'],:]
            np_x = np.rot90(np_x, 3)
            np_y = np.rot90(np_y, 3)
        elif example['zero'] == 'down':
            np_x = np_x[obj_info['y'] + obj_info['h']:,:]
            np_y = np_y[obj_info['y'] + obj_info['h']:,:]
            np_x = np.rot90(np_x, 1)
            np_y = np.rot90(np_y, 1)
        elif example['zero'] == 'left':
            np_x = np_x[:,:obj_info['x']]
            np_y = np_y[:,:obj_info['x']]
            np_x = np.rot90(np_x, 2)
            np_y = np.rot90(np_y, 2)
        elif example['zero'] == 'right':
            np_x = np_x[:, obj_info['x'] + obj_info['w']:]
            np_y = np_y[:, obj_info['x'] + obj_info['w']:]
        
        end_width = 0
        for idx, val in enumerate(np_x[0]):
            if list_check(val,[0,0,0]):
                end_width = idx
                break
        img_info = np.array( [0,0, len(np_x[0]), len(np_x) ], dtype=np.int32)
        
        '''
        tmp_x = np_x
        tmp_y = np_y
        
        zero_x = np.zeros([config.max_size * 3], dtype=np.int32)
        zero_y = np.zeros([config.max_size * 3], dtype=np.int32)
        np_x = np.reshape(np_x, [-1])
        np_y = np.reshape(np_y, [-1])
        zero_x[:len(np_x)] = np_x
        zero_y[:len(np_y)] = np_y
        np_x = zero_x
        np_y = zero_y     
        print('x_location', example['X'])
        print('y_location', example['Y'])
        print('x_size',np.shape(tmp_x))
        print('img_info', img_info)
        '''
        features = tf.train.Features(feature={
            # 실제 이미지 Location
            "id" : tf.train.Feature(int64_list=tf.train.Int64List(value=[example['id']])),
            # 생성할 부분만 잘린 Image Pixel
            "c_image" : tf.train.Feature(bytes_list=tf.train.BytesList(value=[np_x.tostring()])),
            "r_image" : tf.train.Feature(bytes_list=tf.train.BytesList(value=[np_y.tostring()])),
            "img_info" : tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_info.tostring()])),
            "end_width" : tf.train.Feature(int64_list=tf.train.Int64List(value=[end_width]))
            })
        record = tf.train.Example(features=features)
        writer.write(record.SerializeToString())
    writer.close()

In [8]:
def prepro(config):
    print('Preprocessing ... ')
    train_dir = config.train_dir
    dev_dir = config.dev_dir
    test_dir = config.test_dir
    
    # prepro 모듈 동작 시나리오

    input_tf = make_tf_record(train_dir,"Training data")
    produce_tfrecord(input_tf)
    '''
    make_tf_record(dev_dir,"Dev data")
    make_tf_record(test_dir,"Test data")
    '''

## END : Prepro
---

In [ ]:
################# Only Jupyter Notebook
tf.app.flags._global_parser = _argparse.ArgumentParser()
#####################
config = flags.FLAGS
'''
if config.mode == 'prepro':
    tmp = prepro(config)
elif config.mode == 'train':
    train(config)
'''
print('=== Finish ===')


---
## START : Util

In [9]:
def get_record_parser(config):
    def parse(example):
        features = tf.parse_single_example(example, 
                                          features={
                                              "c_image" : tf.FixedLenFeature([], tf.string),
                                              "r_image" : tf.FixedLenFeature([], tf.string),
                                              "img_info" : tf.FixedLenFeature([], tf.string),
                                              "end_width": tf.FixedLenFeature([], tf.int64),
                                              "id" : tf.FixedLenFeature([], tf.int64)
                                          })
        c_image = tf.decode_raw(features["c_image"], tf.int32)
        r_image = tf.decode_raw(features["r_image"], tf.int32)
        img_info = tf.decode_raw(features["img_info"], tf.int32)
        end_width = features['end_width']
        id_ = features['id']
        return c_image, r_image, img_info, end_width, id_
    return parse 

In [10]:
def get_batch_dataset(record_file, parser, config):
    num_threads = tf.constant(config.num_threads, dtype=tf.int32)
    dataset = tf.data.TFRecordDataset(record_file).map(parser, num_parallel_calls=num_threads).shuffle(config.capacity).repeat()
    dataset = dataset.batch(config.batch_size)
    return dataset

## END : Util
---

---
## START : Model

In [ ]:
class Model(object):
    
    def __init__(self, config, batch):
        #### Input Data 부분
        self.config = config
        self.global_step = tf.get_variable('global_step', shape=[], dtype=tf.int32,
                                           initializer=tf.constant_initializer(0), trainable=False)
        
        self.c_image, self.r_image, self.img_info, self.end_width, self.id_ = batch.get_next()
        self.c_image = tf.squeeze(self.c_image)
        self.r_image = tf.squeeze(self.r_image)
        self.img_info = tf.squeeze(self.img_info)
        self.end_width = tf.squeeze(self.end_width)
        self.id_ = tf.squeeze(self.id_)
        
        self.c_image = tf.reshape(self.c_image, [self.img_info[3], self.img_info[2], 3])
        self.r_image = tf.reshape(self.r_image, [self.img_info[3], self.img_info[2], 3])
        self.is_train = tf.get_variable("is_train", shape=[], dtype=tf.bool, trainable=False)
        
        #### Cutting & Copy Data 부분
        enc_seq_len = tf.divide(self.end_width, self.config.block_size)
        enc_seq_len = tf.floor(enc_seq_len) # Encoding 부분은 버림한다.
        
        dec_seq_len = tf.divide( self.img_info[2] - self.end_width, self.config.block_size )
        dec_seq_len = tf.add(tf.ceil(dec_seq_len), 1) # Decoding 부분은 올림한다.
        
        enc_img = tf.slice(self.c_image, 
                           [0,0,0], 
                           [self.img_info[3], enc_seq_len * self.config.block_size, 3])
        dec_img = tf.slice(self.c_image, 
                           [0, self.end_width, 0], 
                           [self.img_info[3], dec_seq_len * self.config.block_size, 3 ])
        
        #### Feature Extracting 부분
        
        enc_img = tf.slic
        #### Partitioning Data 부분
        
        enc_cell = tf.nn.rnn_cell.BasicLSTMCell(self.config.hidden)
        enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
        outputs, dec_states = tf.nn.dynamic_rnn()
        
        #### Feature Extracting 부분
        
        
        
        #### Seq2Seq 부분 
        
        #### Deconvolution 부분.
        
        
        # 이게 뭐지 ???????????????????????????
        

## END : Model
---

---
## START : Train

In [ ]:
print("Building Model ... ")

print("Setting Batch system ... ")
parser = get_record_parser(config)
train_dataset = get_batch_dataset(config.train_record_file, parser, config)
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle( 
    handle, train_dataset.output_types, train_dataset.output_shapes)
train_iterator = train_dataset.make_one_shot_iterator()
model = Model(config, iterator)
print('Finish.')

In [15]:
sess_config = tf.ConfigProto(allow_soft_placement=True)
loss_save = 100.0
patience = 0
lr = config.init_lr

with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(config.train_log_dir)
    saver = tf.train.Saver()
    train_handle = sess.run(train_iterator.string_handle())
    sess.run(tf.assign(model.is_train, tf.constant(True, dtype=tf.bool)))
    
    for _ in tqdm(range(1, config.num_steps + 1)):
        global_step = sess.run(model.global_step) + 1
        c_image, img_info, end_width= sess.run([model.c_image, model.img_info, model.end_width], feed_dict={handle: train_handle})
        print('========================')
        print(img_info)
        print(np.shape(c_image))
        print(end_width)
        print('========================')

  0%|          | 5/10000 [00:00<11:06, 15.00it/s]

[  0   0 181 931]
(931, 181, 3)
116
[  0   0 181 664]
(664, 181, 3)
105
[  0   0 181 461]
(461, 181, 3)
124
[  0   0 376 598]
(598, 376, 3)
134
[  0   0 181 547]
(547, 181, 3)
105
[  0   0 376 540]
(540, 376, 3)
106
[  0   0 376 650]
(650, 376, 3)
293
[  0   0 181 673]
(673, 181, 3)
106


  0%|          | 12/10000 [00:00<07:37, 21.84it/s]

[  0   0 376 497]
(497, 376, 3)
164
[  0   0 181 609]
(609, 181, 3)
122
[  0   0 181 398]
(398, 181, 3)
121
[  0   0 181 633]
(633, 181, 3)
128
[  0   0 376 496]
(496, 376, 3)
294
[  0   0 330 619]
(619, 330, 3)
179


  0%|          | 19/10000 [00:00<06:49, 24.38it/s]

...

[  0   0 226 520]
(520, 226, 3)
108
[  0   0 226 712]
(712, 226, 3)
150
[  0   0 367 463]
(463, 367, 3)
298
[  0   0 367 491]
(491, 367, 3)
177
[  0   0 367 463]
(463, 367, 3)
303
[  0   0 226 699]
(699, 226, 3)
120
[  0   0 367 336]
(336, 367, 3)
210
[  0   0 367 444]
(444, 367, 3)
169
[  0   0 270 395]
(395, 270, 3)
124
[  0   0 270 356]
(356, 270, 3)
197


 85%|████████▌ | 8528/10000 [03:54<00:40, 36.42it/s]

[  0   0 367 358]
(358, 367, 3)
283
[  0   0 367 543]
(543, 367, 3)
311
[  0   0 270 495]
(495, 270, 3)
177


KeyboardInterrupt: 

 85%|████████▌ | 8528/10000 [04:10<00:43, 34.11it/s]

## END : Train
---